# Healthcare Virtual Assistant – Intent Detection and Dialogue Generation

This notebook simulates a real-world healthcare assistant that can:
- Classify user intents using embedding similarity
- Generate responses using a Small Language Model (FLAN-T5-Small)
- Visualize distribution of intents
- Simulate a real-time conversation

**Model Stack:**  
- Sentence-BERT (MiniLM) for intent classification  
- FLAN-T5-Small for dialogue response generation  


In [ ]:

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import random
import time

# SECTION 1: Load and Prepare Data
data = {
    'user_input': [
        'What are the symptoms of diabetes?',
        'I need to book an appointment with a cardiologist',
        'Tell me about high blood pressure',
        'Can I take ibuprofen with my current medication?',
        'I need help with managing asthma',
        'Where can I find a dermatologist?',
        'I feel chest pain, what should I do?',
        'Is it safe to take this vaccine?',
        'I have a fever for 3 days',
        'What should I eat if I have high cholesterol?'
    ]
}
df = pd.DataFrame(data)

# SECTION 2: Load Pre-trained Models
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# SECTION 3: Define Intent Classes
intents = {
    "symptom_check": ["what are the symptoms of", "i feel sick", "i have a fever", "i feel pain", "describe symptoms"],
    "book_appointment": ["book appointment", "schedule doctor", "need a doctor", "see a specialist"],
    "disease_info": ["tell me about", "what is", "explain disease"],
    "medication_advice": ["can i take", "medication interaction", "safe to take"],
    "vaccine_info": ["is it safe", "vaccine reaction", "vaccine information"],
    "diet_advice": ["what should i eat", "diet for", "healthy food", "food suggestion"]
}

intent_sentences = [i for samples in intents.values() for i in samples]
intent_labels = [label for label, samples in intents.items() for _ in samples]
intent_embeddings = embedder.encode(intent_sentences)

# SECTION 4: Inference Functions
def classify_intent(user_input):
    query_embedding = embedder.encode([user_input])
    similarities = cosine_similarity(query_embedding, intent_embeddings)
    best_match_idx = similarities[0].argmax()
    confidence = similarities[0][best_match_idx]
    predicted_intent = intent_labels[best_match_idx]
    return predicted_intent, confidence

def generate_response(user_input, intent):
    prompt = f"Patient asked: {user_input}. Provide a helpful and safe response related to {intent}."
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    output_ids = model.generate(input_ids, max_length=80)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# SECTION 5: Run Assistant
results = []
for i, row in df.iterrows():
    input_text = row['user_input']
    intent, confidence = classify_intent(input_text)
    response = generate_response(input_text, intent)
    results.append({
        'Input': input_text,
        'Intent': intent,
        'Confidence': round(confidence, 2),
        'Response': response
    })
    time.sleep(0.5)

result_df = pd.DataFrame(results)
result_df


In [ ]:

# SECTION 6: Visualizations
plt.figure(figsize=(8,5))
intent_counts = result_df['Intent'].value_counts()
intent_counts.plot(kind='bar', color='skyblue')
plt.title("Detected Intent Distribution")
plt.ylabel("Frequency")
plt.xlabel("Intent")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:

# SECTION 7: Simulated Real-time Assistant
def chat_with_assistant():
    print("Healthcare Assistant: Hello! How can I assist you today?\n(Type 'exit' to quit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        intent, conf = classify_intent(user_input)
        response = generate_response(user_input, intent)
        print(f"Assistant ({intent}, {round(conf,2)}): {response}\n")

# Uncomment to try in a terminal or interactive environment
# chat_with_assistant()
